In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sys
sys.path.append('..')

In [15]:
from autoplan.trainer import ClassifierTrainer, option_combinations
from autoplan.dataset import PrelabeledDataset, build_synthetic_dataset, TrainVal
from autoplan.generator import ProgramGenerator
from autoplan.vis import plot_accuracy, plot_cm, plot_loss
from autoplan.token import TokenType, PyretTokenizer, OCamlTokenizer
from scripts.rainfall_ingest import ingest_dataset

from grammars.rainfall.ocaml import Program
from grammars.rainfall.labels import GeneralRainfallLabels

from tqdm import tqdm_notebook as tqdm
import pandas as pd
import torch
import os
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import pickle
# import seaborn as sns

# device = torch.device('cpu')
device = torch.device('cuda:3')
REPO_DIR = os.path.expanduser('~/autoplan')
path = os.path.expanduser('~/autoplan/data/pickle')

In [16]:
dataset_name = 'T1'
student_dataset = ingest_dataset(dataset_name)

Skipped 16 programs


In [19]:
sample_size = [
    ('N', [1000])
]

# k is the number of datasets generated for each sample size
k = 5

unique_datasets = [
    (opts, build_synthetic_dataset(
                GeneralRainfallLabels,
                tokenizer=OCamlTokenizer(),
                generator=ProgramGenerator(grammar=Program(), adaptive=True),
                vocab_index=student_dataset.vocab_index,
                unique=True,
                **opts))
    for opts in option_combinations(sample_size)
        for i in range(k)
]

duplicate_datasets = [
    (opts, build_synthetic_dataset(
                GeneralRainfallLabels,
                tokenizer=OCamlTokenizer(),
                generator=ProgramGenerator(grammar=Program(), adaptive=True),
                vocab_index=student_dataset.vocab_index,
                unique=False,
                **opts))
    for opts in option_combinations(sample_size)
        for i in range(k)
]

Generating programs...
Generated 1000 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 1000 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 1000 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 1000 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 1000 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 609 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 636 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 616 unique programs.
Tokenizing programs...
Building dataset metadata...
Generating programs...
Generated 612 unique programs.
Tokenizing programs...
Building dataset metadata...


In [20]:
model_options = [
    ('dataset', unique_datasets),
    ('model', [nn.GRU, nn.LSTM]),
    ('hidden_size', [128, 512, 2048]),
    ('embedding_size', [128, 512, 2048]),
]

all_evals = [
    (opts, ClassifierTrainer.crossval(opts['dataset'][1], epochs=100, device=device,
                                      split=TrainVal(opts['dataset'][1], student_dataset), 
                                      model_opts={k: v for k, v in opts.items() if k != 'dataset'}))
    for opts in tqdm(option_combinations(model_options))
]

In [21]:
sorted([{
    'params': k, 
    'accuracy': np.mean(v['accuracy'])
} for k, v in all_evals], key=lambda t: -t['accuracy'])

[{'params': {'embedding_size': 512,
   'hidden_size': 512,
   'model': torch.nn.modules.rnn.LSTM,
   'dataset': ({'N': 1000},
    SyntheticDataset(dataset=<autoplan.dataset.ProgramDataset object at 0x7fccd56016a0>, vocab_size=63, vocab_index={('LET', ''): 0, ('REC', ''): 1, 'LIDENT': 2, ('LPAREN', ''): 3, ('COLON', ''): 4, ('RPAREN', ''): 5, ('EQUAL', ''): 6, ('MATCH', ''): 7, ('COMMA', ''): 8, ('WITH', ''): 9, ('BAR', ''): 10, ('LBRACKET', ''): 11, ('RBRACKET', ''): 12, ('FLOAT', '0.'): 13, ('MINUSGREATER', ''): 14, 'STRING': 15, ('MINUS', ''): 16, ('FLOAT', '999.'): 17, ('COLONCOLON', ''): 18, ('UNDERSCORE', ''): 19, ('IF', ''): 20, ('INFIXOP0', '>='): 21, ('THEN', ''): 22, ('PLUSDOT', ''): 23, ('FLOAT', '1.'): 24, ('ELSE', ''): 25, ('STAR', ''): 26, ('IN', ''): 27, ('INFIXOP3', '/.'): 28, ('INT', '0'): 29, ('INFIXOP3', '/'): 30, ('INT', '999'): 31, ('GREATER', ''): 32, ('INT', '1'): 33, ('PLUS', ''): 34, ('LESS', ''): 35, ('WHEN', ''): 36, ('INFIXOP0', '!='): 37, ('SEMI', ''): 38, (

In [23]:
path = os.path.expanduser('~/autoplan/data/pickle_data/1000_unique')
pickle.dump(unique_datasets, open(path, 'wb'))
path = os.path.expanduser('~/autoplan/data/pickle_data/1000_duplicate')
pickle.dump(duplicate_datasets, open(path, 'wb'))

In [24]:
path = os.path.expanduser('~/autoplan/data/pickle_data/1000_unique_evals')
pickle.dump(all_evals, open(path, 'wb'))

In [ ]:
print(data[0][1]['loss'])
print('='*10)
print(data[0][1]['accuracy'])